In [1]:
from datasets import load_dataset

ds = load_dataset("restor/tcd")
ds

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'height', 'width', 'annotation', 'oam_id', 'license', 'biome', 'crs', 'bounds', 'validation_fold', 'biome_name', 'lat', 'lon', 'segments', 'meta', 'coco_annotations'],
        num_rows: 4169
    })
    test: Dataset({
        features: ['image_id', 'image', 'height', 'width', 'annotation', 'oam_id', 'license', 'biome', 'crs', 'bounds', 'validation_fold', 'biome_name', 'lat', 'lon', 'segments', 'meta', 'coco_annotations'],
        num_rows: 439
    })
})

In [11]:
import matplotlib.pyplot as plt
from PIL import Image
first_image = ds['train'][0]['image']

# Display the image using PIL
first_image.show()

# Alternatively, you can use matplotlib to display the image
plt.imshow(first_image)
# plt.axis('off')  # Hide the axes
# plt.show()
first_image = ds['train'][0]['annotation']

# Display the image using PIL
first_image.show()

# Alternatively, you can use matplotlib to display the image
plt.imshow(first_image)

NotImplementedError: Subclasses of Dataset should implement __getitem__.

In [ ]:
from PIL import Image

# Check the first few images and annotations
for i in range(min(5, len(ds['train']))):  # Check first 5 examples
    try:
        image = ds['train']['image'][i]
        annotation = ds['train']['annotation'][i]
        
        # Ensure the image and annotation are PIL images
        if isinstance(image, Image.Image) and isinstance(annotation, Image.Image):
            print(f"Example {i}: Image and annotation are valid PIL images.")
        else:
            print(f"Example {i}: Image or annotation is not a PIL image.")
    except Exception as e:
        print(f"Error in example {i}: {e}")

In [3]:
import logging
import os
import sys
import tempfile
from glob import glob

import torch
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot as plt
import monai
from monai.data import ArrayDataset, create_test_image_2d, decollate_batch, DataLoader
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AsDiscrete,
    Compose,
    LoadImage,
    RandRotate90,
    RandSpatialCrop,
    ScaleIntensity,
)
from monai.visualize import plot_2d_or_3d_image



from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.transforms import (Compose, LoadImage, EnsureChannelFirstd, ScaleIntensityd, ToTensor, 
                            RandFlip, RandRotate, Resize)


In [2]:
from torch.utils.data import Dataset, DataLoader
from monai.transforms import Compose, EnsureChannelFirstd, ScaleIntensityd, Resize, ToTensor, Activations, AsDiscrete
import numpy as np
from PIL import Image

class SegmentationDataset(Dataset):
    def __init__(self, images, labels, transforms=None):
        self.images = images  # List of PIL images
        self.labels = labels  # List of PIL labels (annotations)
        self.transforms = transforms
        assert len(self.images) == len(self.labels), "Mismatch in image/label counts"

    def __getitem__(self, index):
        try:
            # Convert PIL images to numpy arrays
            image_np = np.array(self.images[index])
            label_np = np.array(self.labels[index])

            data = {'image': image_np, 'label': label_np}

            if self.transforms:
                data = self.transforms(data)

            return data['image'], data['label']
        except Exception as e:
            # Handle corrupted or invalid data
            print(f"Error loading data at index {index}: {e}")
            # Return a placeholder or skip this example
            return torch.zeros((1, 256, 256)), torch.zeros((1, 256, 256))


    # def __getitem__(self, index):
    #     # Get the image object (it might be a lazy-loaded PIL Image)
    #     image_obj = self.images[index]
    #     label_obj = self.labels[index]
        
    #     # Force the image to load fully (and convert to a new object)
    #     image = image_obj.copy()  # now fully loaded in memory
    #     label = label_obj.copy()
        
    #     # Convert to numpy arrays
    #     image_np = np.array(image)
    #     label_np = np.array(label)

    #     data = {'image': image_np, 'label': label_np}

    #     if self.transforms:
    #         data = self.transforms(data)

    #     return data['image'], data['label']

    def __len__(self):
        return len(self.images)


class SegmentationDataset(Dataset):
    def __init__(self, image_paths, label_paths, transforms=None):
        self.image_paths = image_paths  # List of image file paths
        self.label_paths = label_paths  # List of label file paths
        self.transforms = transforms
        assert len(self.image_paths) == len(self.label_paths), "Mismatch in image/label counts"

    def __getitem__(self, index):
        try:
            # Load images and labels from disk
            image = Image.open(self.image_paths[index])
            label = Image.open(self.label_paths[index])

            # Convert to numpy arrays
            image_np = np.array(image)
            label_np = np.array(label)

            data = {'image': image_np, 'label': label_np}

            if self.transforms:
                data = self.transforms(data)

            return data['image'], data['label']
        except Exception as e:
            # Handle corrupted or invalid data
            print(f"Error loading data at index {index}: {e}")
            # Return a placeholder or skip this example
            return torch.zeros((1, 256, 256)), torch.zeros((1, 256, 256))

    def __len__(self):
        return len(self.image_paths)
# Define transforms (no LoadImage needed)
keys = ['image', 'label']
train_transform = Compose([
    EnsureChannelFirstd(keys=keys, channel_dim="no_channel"),  # Handles HWC -> CHW
    ScaleIntensityd(keys=["image"]),
    Resize(spatial_size=(256, 256)),
    ToTensor()
])

val_transform = Compose([  # Typically same as train for basic preprocessing
    EnsureChannelFirstd(keys=keys, channel_dim="no_channel"),
    ScaleIntensityd(keys=["image"]),
    Resize(spatial_size=(256, 256)),
    ToTensor()
])

# Load dataset and create datasets
# from datasets import load_dataset
# ds = load_dataset("restor/tcd")

# # Training dataset
# train_ds = SegmentationDataset(
#     images=ds['train']['image'],
#     labels=ds['train']['annotation'],
#     transforms=train_transform
# )

# # Validation dataset (using test split as validation)
# val_ds = SegmentationDataset(
#     images=ds['test']['image'],
#     labels=ds['test']['annotation'],
#     transforms=val_transform
# )



# # Limit the dataset to the first 30 images and labels
# train_images = ds['train']['image'][:30]
# train_labels = ds['train']['annotation'][:30]
# test_images = ds['test']['image'][:30]
# test_labels = ds['test']['annotation'][:30]

# # Training dataset
# train_ds = SegmentationDataset(
#     images=train_images,
#     labels=train_labels,
#     transforms=train_transform
# )

# # Validation dataset (using test split as validation)
# val_ds = SegmentationDataset(
#     images=test_images,
#     labels=test_labels,
#     transforms=val_transform
# )

# # Create DataLoaders
# train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
# val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

# # Test the DataLoader
# for batch in train_loader:
#     images, labels = batch
#     print(images.shape, labels.shape)
#     break

In [1]:
from datasets import load_dataset
ds = load_dataset("restor/tcd")



In [3]:


train_ds = SegmentationDataset(
    images=ds['train']['image'],
    labels=ds['train']['annotation'],
    transforms=train_transform
)

# Validation dataset (using test split as validation)
val_ds = SegmentationDataset(
    images=ds['test']['image'],
    labels=ds['test']['annotation'],
    transforms=val_transform
)

OSError: -9

In [5]:
import torch
from torch.utils.data import IterableDataset, DataLoader
from datasets import load_dataset
from monai.transforms import Compose, EnsureChannelFirstd, ScaleIntensityd, ToTensor
import numpy as np
from PIL import Image
import logging

# Configure logging to catch worker errors
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set multiprocessing strategy (critical for Windows)
torch.multiprocessing.set_sharing_strategy('file_system')

# Define transforms
keys = ['image', 'label']
train_transform = Compose([
    EnsureChannelFirstd(keys=keys, channel_dim="no_channel"),
    ScaleIntensityd(keys=["image"]),
    ToTensor()
])

class SafeHFIterableDataset(IterableDataset):
    def __init__(self, hf_dataset, transform=None, buffer_size=1000):
        self.hf_dataset = hf_dataset
        self.transform = transform
        self.buffer_size = buffer_size
        self._shard = None

    def _init_shard(self):
        """Initialize dataset shard for this worker"""
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is not None:
            self._shard = self.hf_dataset.shard(
                num_shards=worker_info.num_workers,
                index=worker_info.id,
                contiguous=True
            )
        else:
            self._shard = self.hf_dataset

    def __iter__(self):
        self._init_shard()
        buffer = []
        
        for example in self._shard:
            try:
                # Stream and process one item at a time
                image = example['image'].convert("L").resize((256, 256), Image.BILINEAR)
                label = example['annotation'].convert("L").resize((256, 256), Image.NEAREST)

                data = {
                    'image': np.array(image),
                    'label': np.array(label)
                }

                if self.transform:
                    data = self.transform(data)

                buffer.append((data['image'], data['label']))
                
                # Maintain buffer to prevent empty queues
                if len(buffer) >= self.buffer_size:
                    yield from buffer
                    buffer = []
                    
            except Exception as e:
                logger.error(f"Skipping corrupted example: {str(e)[:200]}")
                continue

        # Yield remaining items
        if buffer:
            yield from buffer

# Load datasets with explicit PIL format
train_ds_hf = load_dataset("restor/tcd", split="train", streaming=True).with_format("pil")
val_ds_hf = load_dataset("restor/tcd", split="test", streaming=True).with_format("pil")

# Create datasets
train_ds = SafeHFIterableDataset(train_ds_hf, transform=train_transform)
val_ds = SafeHFIterableDataset(val_ds_hf, transform=train_transform)

# Create DataLoaders with conservative settings
loader_config = {
    'batch_size': 8,  # Reduced from 16
    'num_workers': 2,  # Start with 1 if issues persist
    'persistent_workers': True,
    'prefetch_factor': None,  # Let DataLoader manage prefetching
    'pin_memory': True
}

train_loader = DataLoader(train_ds, **loader_config)
val_loader = DataLoader(val_ds, **loader_config)

In [6]:

# Create DataLoaders
# train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
# val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)
torch.multiprocessing.freeze_support()
torch.multiprocessing.set_start_method('spawn', force=True)
# Metrics and post-processing (unchanged)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
# create UNet, DiceLoss and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = monai.networks.nets.UNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64),
    strides=(2, 2),
).to(device)
loss_function = monai.losses.DiceLoss(sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-3)
# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = list()
metric_values = list()
writer = SummaryWriter()



# train_loader, val_loader = create_train_val_dataloaders(images_dir=images_dir, labels_dir=labels_dir)

print("Trainloader:")
print(train_loader)

for epoch in range(10):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{10}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        print(f"epoch {epoch + 1}/{10} batch = ")
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            val_images = None
            val_labels = None
            val_outputs = None
            for val_data in val_loader:
                val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
                roi_size = (96, 96)
                sw_batch_size = 4
                val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)
                val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                # compute metric for current iteration
                dice_metric(y_pred=val_outputs, y=val_labels)
            # aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # reset the status for next validation round
            dice_metric.reset()
            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "best_metric_model_segmentation2d_array.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                    epoch + 1, metric, best_metric, best_metric_epoch
                )
            )
            writer.add_scalar("val_mean_dice", metric, epoch + 1)
            # plot the last model output as GIF image in TensorBoard with the corresponding image and label
            plot_2d_or_3d_image(val_images, epoch + 1, writer, index=0, tag="image")
            plot_2d_or_3d_image(val_labels, epoch + 1, writer, index=0, tag="label")
            plot_2d_or_3d_image(val_outputs, epoch + 1, writer, index=0, tag="output")
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()
# torch.save(model.state_dict(), "trained_model.pth")

Trainloader:
----------
epoch 1/10


RuntimeError: DataLoader worker (pid(s) 25324, 15656) exited unexpectedly